In [1]:
from math import sqrt
import copy
import  traceback
import shutil
import random

import numpy as np  # linear algebra
import pydicom
import os
import matplotlib.pyplot as plt
import cv2
from pydicom.uid import UID
from PIL import Image
from tqdm import tqdm

In [2]:
def load_scan(path):
    slices = [] #slices = [pydicom.dcmread(path + '/' + s) for s in filter(lambda x: x.endswith('.dcm'), os.listdir(path))]
    for s in os.listdir(path):
        if not s.endswith('.dcm'):
            continue
        sl = pydicom.dcmread(os.path.join(path, s))
        try:
            sl_p = sl.pixel_array
        except:
            traceback.print_exc()
            print(f'\tDelete {os.path.join(path, s)}')
            os.remove(os.path.join(path, s))
        else:
            slices.append(sl)
    slices.sort(key=lambda x: float(x.InstanceNumber))
    return slices

In [3]:
lower_b, upper_b = -100, 500

## 1.阳性数据

In [ ]:
# 将dcm文件根据窗宽窗位转化为png图片
def generate_image(input_folder):
    for patient in os.listdir(input_folder):
        if os.path.isfile(os.path.join(input_folder, patient)):
            continue
        print(f'****Processing {patient}****')
        for scan in os.listdir(os.path.join(input_folder, patient)):
            if scan != '1':
                continue
            name = patient #name = patient.split('-')[0]
            image_path = os.path.join(input_folder, patient, scan, f'images_{lower_b}_{upper_b}')
            if os.path.exists(image_path):
                shutil.rmtree(image_path)
            os.mkdir(image_path)

            ct = load_scan(os.path.join(input_folder, patient, scan))

            for i in range(len(ct)):
                img = ct[i].pixel_array.astype(np.int16)
                intercept = ct[i].RescaleIntercept
                slope = ct[i].RescaleSlope
                if slope != 1:
                    img = (slope * img.astype(np.float64)).astype(np.int16)
                img += np.int16(intercept)
                img = np.clip(img, lower_b, upper_b)
                img = ((img-lower_b)/(upper_b-lower_b)*255).astype(np.uint8)
                img = Image.fromarray(img)
                img.save(os.path.join(image_path, f'{name}_{i:04d}.png'))

# generate_image('/nfs3-p1/zsxm/dataset/2021-9-8/')
# generate_image('/nfs3-p1/zsxm/dataset/2021-9-13/')
# generate_image('/nfs3-p1/zsxm/dataset/2021-9-19/')
generate_image('/nfs3-p1/zsxm/dataset/2021-9-28/')

In [ ]:
# 将各个病例中的png图片文件夹统一移动到一起供yolov5检测
def move_together_for_detect(input_folder, dst_path):
    if not os.path.exists(dst_path):
        os.mkdir(dst_path)
    root_name = input_folder.split('/')[-1] if input_folder.split('/')[-1] != '' else input_folder.split('/')[-2]
    dst_path = os.path.join(dst_path, root_name)

    for patient in os.listdir(input_folder):
        if os.path.isfile(os.path.join(input_folder, patient)):
            continue
        print(f'****Processing {patient}****')
        name = patient #name = patient.split('-')[0]
        if os.path.exists(os.path.join(dst_path, name)):
            print(f"\tremove {os.path.join(dst_path, name)}")
            shutil.rmtree(os.path.join(dst_path, name))

        try:
            shutil.copytree(os.path.join(input_folder, patient, '1', f'images_{lower_b}_{upper_b}'), os.path.join(dst_path, name))
        except:
            traceback.print_exc()

# move_together_for_detect('/nfs3-p1/zsxm/dataset/2021-9-8/', '/nfs3-p1/zsxm/dataset/9_detect/')
# move_together_for_detect('/nfs3-p1/zsxm/dataset/2021-9-13/', '/nfs3-p1/zsxm/dataset/9_detect/')
# move_together_for_detect('/nfs3-p1/zsxm/dataset/2021-9-19/', '/nfs3-p1/zsxm/dataset/9_detect/')
move_together_for_detect('/nfs3-p1/zsxm/dataset/2021-9-28/', '/nfs3-p1/zsxm/dataset/9_detect/')

In [ ]:
# 用在更换窗宽窗位的时候，因为已经经过yolov5的检测生成了标签，故直接移动图片到images文件夹下即可
def directly_move_to_image_label(input_folder, dst_path):
    if not os.path.exists(dst_path):
        os.mkdir(dst_path)

    for patient in os.listdir(input_folder):
        if os.path.isfile(os.path.join(input_folder, patient)):
            continue
        print(f'****Processing {patient}****')
        name = patient #name = patient.split('-')[0]
        image_path = os.path.join(dst_path, name, 'images')
        if os.path.exists(image_path):
            print(f"\tremove {image_path}")
            shutil.rmtree(image_path)

        try:
            shutil.copytree(os.path.join(input_folder, patient, '1', f'images_{lower_b}_{upper_b}'), image_path)
        except:
            traceback.print_exc()
            
directly_move_to_image_label('/nfs3-p1/zsxm/dataset/2021-9-8/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive')
directly_move_to_image_label('/nfs3-p1/zsxm/dataset/2021-9-13/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive')
directly_move_to_image_label('/nfs3-p1/zsxm/dataset/2021-9-19/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive_new')

In [ ]:
# 将移动到待测文件夹中的原图移动到images文件夹
def move_ori_image_to_sigle_folder(input_path, dst_path):
    if not os.path.exists(dst_path):
        os.mkdir(dst_path)
    for patient in os.listdir(input_path):
        print(f'****Processing {patient}****')
        if not os.path.exists(os.path.join(dst_path, patient)):
            os.mkdir(os.path.join(dst_path, patient))
        image_path = os.path.join(dst_path, patient, 'images')
        if os.path.exists(image_path):
            shutil.rmtree(image_path)
        os.mkdir(image_path)
        for img in os.listdir(os.path.join(input_path, patient)):
            if os.path.isfile(os.path.join(input_path, patient, img)):
                shutil.move(os.path.join(input_path, patient, img), os.path.join(image_path, img))
        os.rmdir(os.path.join(input_path, patient))
    os.rmdir(input_path)
    
# move_ori_image_to_sigle_folder('/nfs3-p1/zsxm/dataset/9_detect/2021-9-8/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive')
# move_ori_image_to_sigle_folder('/nfs3-p1/zsxm/dataset/9_detect/2021-9-13/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive')
# move_ori_image_to_sigle_folder('/nfs3-p1/zsxm/dataset/9_detect/2021-9-19/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive_new')
move_ori_image_to_sigle_folder('/nfs3-p1/zsxm/dataset/9_detect/2021-9-28/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive_new2')

In [ ]:
# 将yolov5的检测结果（标签和带bounding box的图像）移动到相应文件夹
def move_detect_result_to_folder(input_path, output_path):
    for patient in os.listdir(input_path):
        print(f'Processing {patient}')
        image_path = os.path.join(output_path, patient, 'pred_images')
        if os.path.exists(image_path):
            shutil.rmtree(image_path)
        os.mkdir(image_path)
        for file in os.listdir(os.path.join(input_path, patient)):
            if os.path.isfile(os.path.join(input_path, patient, file)):
                shutil.move(os.path.join(input_path, patient, file), os.path.join(image_path, file))
            elif os.path.isdir(os.path.join(input_path, patient, file)):
                if os.path.exists(os.path.join(output_path, patient, file)):
                    shutil.rmtree(os.path.join(output_path, patient, file))
                shutil.move(os.path.join(input_path, patient, file), os.path.join(output_path, patient, file))
        os.rmdir(os.path.join(input_path, patient))
    os.rmdir(input_path)
                
# move_detect_result_to_folder('/nfs3-p1/zsxm/dataset/9_detect/pred/2021-9-8/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive')
# move_detect_result_to_folder('/nfs3-p1/zsxm/dataset/9_detect/pred/2021-9-13/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive')
# move_detect_result_to_folder('/nfs3-p1/zsxm/dataset/9_detect/pred/2021-9-19/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive_new')
move_detect_result_to_folder('/nfs3-p1/zsxm/dataset/9_detect/pred/2021-9-28/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive_new2')

In [ ]:
# 切出主动脉
def find_coordinate(height, width, label_file, aorta):
    with open(label_file, 'r') as f:
        lines = f.readlines()
    assert len(lines) <= 2, f'label.txt应该存储不多于2个label：{label_file.split("/")[-1]}'
    if len(lines) == 1:
        assert aorta == 'j', f'如果只有一个label那么此时应为降主动脉, 但实际为{aorta}：{label_file.split("/")[-1]}'
        corr = list(map(lambda x: float(x), lines[0].split()))
        x, y, w, h = corr[1], corr[2], corr[3], corr[4]
        assert 0.25 < x < 0.75 and 0.25 < y < 0.75, f'边界框中心({x}, {y})出界：{label_file.split("/")[-1]}'
    else:
        corr1, corr2 = list(map(lambda x: float(x), lines[0].split())), list(map(lambda x: float(x), lines[1].split()))
        assert 0.25 < corr1[1] < 0.75 and 0.25 < corr1[2] < 0.75, f'边界框1中心({corr1[1]}, {corr1[2]})出界：{label_file.split("/")[-1]}'
        assert 0.25 < corr2[1] < 0.75 and 0.25 < corr2[2] < 0.75, f'边界框2中心({corr2[1]}, {corr2[2]})出界：{label_file.split("/")[-1]}'
        if aorta == 's':
            x, y, w, h = (corr1[1], corr1[2], corr1[3], corr1[4]) if corr1[2] < corr2[2] else (corr2[1], corr2[2], corr2[3], corr2[4])
        elif aorta == 'j':
            x, y, w, h = (corr1[1], corr1[2], corr1[3], corr1[4]) if corr1[2] > corr2[2] else (corr2[1], corr2[2], corr2[3], corr2[4])
        else:
            raise Exception(f'aorta 应该为"s"或"j"其中之一: {label_file.split("/")[-1]}')
    w, h = int(width*w), int(height*h)
    w, h = max(w, h), max(w, h)
    return int(width*x-w/2), int(height*y-h/2), int(width*x+w/2+1), int(height*y+h/2+1)

def crop_images(input_path, error_patient_list):
    for patient in sorted(os.listdir(input_path)):
        print(f'******Processing {patient}******')
        image_path = os.path.join(input_path, patient, 'images')
        label_path = os.path.join(input_path, patient, 'labels')
        crop_path = os.path.join(input_path, patient, 'crops')
        if os.path.exists(crop_path):
            shutil.rmtree(crop_path)
        os.mkdir(crop_path)
        pl = patient.lower().split('-')
        for i, s in enumerate(pl):
            if i!=0 and s.startswith('im'):
                pl[i] = s.replace('im', '')
        crop_flag = True
        for i, s in enumerate(pl):
            if s != 's' and s != 'j':
                continue
            start, end = int(pl[i+1])-1, int(pl[i+2])
            for j in range(start, end):
                img = Image.open(os.path.join(image_path, f'{patient}_{j:04d}.png'))
                img = np.array(img)
                try:
                    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{j:04d}.txt'), s)
                except:
                    traceback.print_exc()
                    crop_flag = False
                else:#if crop_flag:
                    crop = img[y1:y2, x1:x2]
                    crop = Image.fromarray(crop)
                    crop.save(os.path.join(crop_path, f'{patient}_{s}_{j:04d}.png'))
        if not crop_flag:
            #print('Delete crop_path')
            #shutil.rmtree(crop_path)
            error_patient_list.append(patient)

epl1 = []
epl2 = []
epl3 = []
# crop_images('/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive', epl1)
# print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
# print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
# print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
# print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
# print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
# crop_images('/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive_new', epl2)
# print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
# print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
# print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
# print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
# print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
crop_images('/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive_new2', epl3)

In [ ]:
# print(len(epl1))
# print(epl1)
# print(len(epl2))
# print(epl2)
print(len(epl3))
print(epl3)

In [ ]:
# # 将切出的主动脉移动到一起
# def move_to_classify(input_root, output_path):
#     if not os.path.exists(output_path):
#         os.makedirs(output_path)
    
#     for patient in os.listdir(input_root):
#         if os.path.isfile(os.path.join(input_root, patient)):
#             continue
#         print(f'****Processing {patient}****')
#         for file in os.listdir(os.path.join(input_root, patient, 'crops')):
#             shutil.copy(os.path.join(input_root, patient, 'crops', file), output_path)

# move_to_classify('/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive/', f'/nfs3-p1/zsxm/dataset/aorta_classify_ct_{lower_b}_{upper_b}/1')
# move_to_classify('/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive_new/', f'/nfs3-p1/zsxm/dataset/aorta_classify_ct_{lower_b}_{upper_b}/1')

## 2.阴性数据

In [11]:
# 将dcm文件根据窗宽窗位转化为png图片
def generate_image(input_folder):
    for patient in os.listdir(input_folder):
        if os.path.isfile(os.path.join(input_folder, patient)):
            continue
        print(f'****Processing {patient}****')
        for scan in os.listdir(os.path.join(input_folder, patient)):
            if scan != '1':
                continue
            name = patient #name = patient.split('-')[0]
            image_path = os.path.join(input_folder, patient, scan, f'images_{lower_b}_{upper_b}')
            if os.path.exists(image_path):
                shutil.rmtree(image_path)
            os.mkdir(image_path)

            ct = load_scan(os.path.join(input_folder, patient, scan))

            for i in range(len(ct)):
                img = ct[i].pixel_array.astype(np.int16)
                intercept = ct[i].RescaleIntercept
                slope = ct[i].RescaleSlope
                if slope != 1:
                    img = (slope * img.astype(np.float64)).astype(np.int16)
                img += np.int16(intercept)
                img = np.clip(img, lower_b, upper_b)
                img = ((img-lower_b)/(upper_b-lower_b)*255).astype(np.uint8)
                img = Image.fromarray(img)
                img.save(os.path.join(image_path, f'{name}_{i:04d}.png'))

#generate_image('/nfs3-p2/zsxm/dataset/2021-9-17-negative/')
generate_image('/nfs3-p2/zsxm/dataset/2021-9-29-negative/')

****Processing tongyanwen-22-27-41-89****
****Processing zhangqingrong-32-43-73-161****


Traceback (most recent call last):
  File "<ipython-input-2-f58982e4f7d0>", line 8, in load_scan
    sl_p = sl.pixel_array
  File "/disk2/zsxm/miniconda3/envs/pytorch/lib/python3.8/site-packages/pydicom/dataset.py", line 835, in __getattr__
    return object.__getattribute__(self, name)
  File "/disk2/zsxm/miniconda3/envs/pytorch/lib/python3.8/site-packages/pydicom/dataset.py", line 1716, in pixel_array
    self.convert_pixel_data()
  File "/disk2/zsxm/miniconda3/envs/pytorch/lib/python3.8/site-packages/pydicom/dataset.py", line 1435, in convert_pixel_data
    self._convert_pixel_data_without_handler()
  File "/disk2/zsxm/miniconda3/envs/pytorch/lib/python3.8/site-packages/pydicom/dataset.py", line 1545, in _convert_pixel_data_without_handler
    raise last_exception
  File "/disk2/zsxm/miniconda3/envs/pytorch/lib/python3.8/site-packages/pydicom/dataset.py", line 1525, in _convert_pixel_data_without_handler
    self._do_pixel_data_conversion(handler)
  File "/disk2/zsxm/miniconda3/envs

	Delete /nfs3-p2/zsxm/dataset/2021-9-29-negative/zhangqingrong-32-43-73-161/1/1-00A1.dcm


In [12]:
# 将各个病例中的png图片文件夹统一移动到一起供yolov5检测
def move_together_for_detect(input_folder, dst_path):
    if not os.path.exists(dst_path):
        os.mkdir(dst_path)
    root_name = input_folder.split('/')[-1] if input_folder.split('/')[-1] != '' else input_folder.split('/')[-2]
    dst_path = os.path.join(dst_path, root_name)

    for patient in os.listdir(input_folder):
        if os.path.isfile(os.path.join(input_folder, patient)):
            continue
        print(f'****Processing {patient}****')
        name = patient #name = patient.split('-')[0]
        if os.path.exists(os.path.join(dst_path, name)):
            print(f"\tremove {os.path.join(dst_path, name)}")
            shutil.rmtree(os.path.join(dst_path, name))

        try:
            shutil.copytree(os.path.join(input_folder, patient, '1', f'images_{lower_b}_{upper_b}'), os.path.join(dst_path, name))
        except:
            traceback.print_exc()

# move_together_for_detect('/nfs3-p2/zsxm/dataset/2021-9-17-negative', '/nfs3-p1/zsxm/dataset/9_detect/')
move_together_for_detect('/nfs3-p2/zsxm/dataset/2021-9-29-negative', '/nfs3-p1/zsxm/dataset/9_detect/')

****Processing tongyanwen-22-27-41-89****
****Processing zhangqingrong-32-43-73-161****
****Processing baimeizhu-27-37-54-136****
****Processing bairuixin-13-18-35-81****
****Processing baodezhong-22-29-39-91****
****Processing baoxialin-23-34-57-158****
****Processing baoyanzu-15-20-31-87****
****Processing caijinfei-34-42-51-155****
****Processing caishiping-18-24-34-85****
****Processing caoyuying-18-22-33-79****
****Processing chenbin-20-26-35-85****
****Processing chenbingrong-31-40-61-153****
****Processing chenchong-22-26-36-86****
****Processing chendonghong-12-17-27-75****
****Processing chengenye-13-18-28-79****
****Processing chengshaoping-19-28-45-132****
****Processing chengshuiping-23-31-48-129****
****Processing chenguoming-28-38-60-144****
****Processing chengyahui-17-22-29-82****
****Processing chenjinfu-11-18-30-80****
****Processing chenjufang-18-27-42-130****
****Processing chenjunbing-22-29-34-89****
****Processing chenjunliang-27-37-58-147****
****Processing chenl

****Processing qiananv-18-23-33-76****
****Processing qianliujiu-9-15-24-75****
****Processing qiuguozhang-27-36-52-136****
****Processing qiujinrong-24-34-58-158****
****Processing qiumeijun-28-36-52-133****
****Processing qiuwei-31-39-53-138****
****Processing qizhikun-29-37-57-130****
****Processing quanjianzhong-24-36-51-138****
****Processing qubo-14-19-29-82****
****Processing renjiali-34-42-52-138****
****Processing renlaifa-19-30-53-153****
****Processing renxiuyuan-18-23-31-79****
****Processing shanziliang-13-19-30-90****
****Processing shanzixian-30-39-59-153****
****Processing shaobingsheng-13-19-27-80****
****Processing shaochuanlai-16-22-36-83****
****Processing shaofeng-23-29-37-95****
****Processing shaominjun-18-24-35-85****
****Processing shenfabiao-29-39-61-135****
****Processing shenguilin-18-25-35-53****
****Processing shenhaiqin-20-28-38-129****
****Processing shenhuinian-21-27-39-89****
****Processing shenshuigen-30-41-56-140****
****Processing shensongping-10-21

****Processing zhuodepeng-9-14-23-78****
****Processing zhuxingfu-14-19-32-82****
****Processing zhuyanjie-20-25-35-89****


In [ ]:
# 用在更换窗宽窗位的时候，因为已经经过yolov5的检测生成了标签，故直接移动图片到images文件夹下即可
def directly_move_to_image_label(input_folder, dst_path):
    if not os.path.exists(dst_path):
        os.mkdir(dst_path)

    for patient in os.listdir(input_folder):
        if os.path.isfile(os.path.join(input_folder, patient)):
            continue
        print(f'****Processing {patient}****')
        name = patient #name = patient.split('-')[0]
        image_path = os.path.join(dst_path, name, 'images')
        if os.path.exists(image_path):
            print(f"\tremove {image_path}")
            shutil.rmtree(image_path)

        try:
            shutil.copytree(os.path.join(input_folder, patient, '1', f'images_{lower_b}_{upper_b}'), image_path)
        except:
            traceback.print_exc()
            
directly_move_to_image_label('/nfs3-p1/zsxm/dataset/2021-9-17-negative/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative')

In [14]:
# 将移动到待测文件夹中的原图移动到images文件夹
def move_ori_image_to_sigle_folder(input_path, dst_path):
    if not os.path.exists(dst_path):
        os.mkdir(dst_path)
    for patient in os.listdir(input_path):
        print(f'****Processing {patient}****')
        if not os.path.exists(os.path.join(dst_path, patient)):
            os.mkdir(os.path.join(dst_path, patient))
        image_path = os.path.join(dst_path, patient, 'images')
        if os.path.exists(image_path):
            shutil.rmtree(image_path)
        os.mkdir(image_path)
        for img in os.listdir(os.path.join(input_path, patient)):
            if os.path.isfile(os.path.join(input_path, patient, img)):
                shutil.move(os.path.join(input_path, patient, img), os.path.join(image_path, img))
        os.rmdir(os.path.join(input_path, patient))
    os.rmdir(input_path)
    
# move_ori_image_to_sigle_folder('/nfs3-p1/zsxm/dataset/9_detect/2021-9-17-negative/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative')
move_ori_image_to_sigle_folder('/nfs3-p1/zsxm/dataset/9_detect/2021-9-29-negative/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2')

****Processing tongyanwen-22-27-41-89****
****Processing zhangqingrong-32-43-73-161****
****Processing baimeizhu-27-37-54-136****
****Processing bairuixin-13-18-35-81****
****Processing baodezhong-22-29-39-91****
****Processing baoxialin-23-34-57-158****
****Processing baoyanzu-15-20-31-87****
****Processing caijinfei-34-42-51-155****
****Processing caishiping-18-24-34-85****
****Processing caoyuying-18-22-33-79****
****Processing chenbin-20-26-35-85****
****Processing chenbingrong-31-40-61-153****
****Processing chenchong-22-26-36-86****
****Processing chendonghong-12-17-27-75****
****Processing chengenye-13-18-28-79****
****Processing chengshaoping-19-28-45-132****
****Processing chengshuiping-23-31-48-129****
****Processing chenguoming-28-38-60-144****
****Processing chengyahui-17-22-29-82****
****Processing chenjinfu-11-18-30-80****
****Processing chenjufang-18-27-42-130****
****Processing chenjunbing-22-29-34-89****
****Processing chenjunliang-27-37-58-147****
****Processing chenl

****Processing qiananv-18-23-33-76****
****Processing qianliujiu-9-15-24-75****
****Processing qiuguozhang-27-36-52-136****
****Processing qiujinrong-24-34-58-158****
****Processing qiumeijun-28-36-52-133****
****Processing qiuwei-31-39-53-138****
****Processing qizhikun-29-37-57-130****
****Processing quanjianzhong-24-36-51-138****
****Processing qubo-14-19-29-82****
****Processing renjiali-34-42-52-138****
****Processing renlaifa-19-30-53-153****
****Processing renxiuyuan-18-23-31-79****
****Processing shanziliang-13-19-30-90****
****Processing shanzixian-30-39-59-153****
****Processing shaobingsheng-13-19-27-80****
****Processing shaochuanlai-16-22-36-83****
****Processing shaofeng-23-29-37-95****
****Processing shaominjun-18-24-35-85****
****Processing shenfabiao-29-39-61-135****
****Processing shenguilin-18-25-35-53****
****Processing shenhaiqin-20-28-38-129****
****Processing shenhuinian-21-27-39-89****
****Processing shenshuigen-30-41-56-140****
****Processing shensongping-10-21

****Processing zhuodepeng-9-14-23-78****
****Processing zhuxingfu-14-19-32-82****
****Processing zhuyanjie-20-25-35-89****


In [15]:
# 将yolov5的检测结果（标签和带bounding box的图像）移动到相应文件夹
def move_detect_result_to_folder(input_path, output_path):
    for patient in os.listdir(input_path):
        print(f'Processing {patient}')
        image_path = os.path.join(output_path, patient, 'pred_images')
        if os.path.exists(image_path):
            shutil.rmtree(image_path)
        os.mkdir(image_path)
        for file in os.listdir(os.path.join(input_path, patient)):
            if os.path.isfile(os.path.join(input_path, patient, file)):
                shutil.move(os.path.join(input_path, patient, file), os.path.join(image_path, file))
            elif os.path.isdir(os.path.join(input_path, patient, file)):
                if os.path.exists(os.path.join(output_path, patient, file)):
                    shutil.rmtree(os.path.join(output_path, patient, file))
                shutil.move(os.path.join(input_path, patient, file), os.path.join(output_path, patient, file))
        os.rmdir(os.path.join(input_path, patient))
    os.rmdir(input_path)
                
# move_detect_result_to_folder('/nfs3-p1/zsxm/dataset/9_detect/pred/2021-9-17-negative/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative')
move_detect_result_to_folder('/nfs3-p1/zsxm/dataset/9_detect/pred/2021-9-29-negative/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2')

Processing tongyanwen-22-27-41-89
Processing zhangqingrong-32-43-73-161
Processing baimeizhu-27-37-54-136
Processing bairuixin-13-18-35-81
Processing baodezhong-22-29-39-91
Processing baoxialin-23-34-57-158
Processing baoyanzu-15-20-31-87
Processing caijinfei-34-42-51-155
Processing caishiping-18-24-34-85
Processing caoyuying-18-22-33-79
Processing chenbin-20-26-35-85
Processing chenbingrong-31-40-61-153
Processing chenchong-22-26-36-86
Processing chendonghong-12-17-27-75
Processing chengenye-13-18-28-79
Processing chengshaoping-19-28-45-132
Processing chengshuiping-23-31-48-129
Processing chenguoming-28-38-60-144
Processing chengyahui-17-22-29-82
Processing chenjinfu-11-18-30-80
Processing chenjufang-18-27-42-130
Processing chenjunbing-22-29-34-89
Processing chenjunliang-27-37-58-147
Processing chenlidong-18-23-32-88
Processing chenlindi-16-22-34-78
Processing chenliuer-23-32-43-140
Processing chenmaojin-14-20-31-90
Processing chenpeisheng-15-20-28-82
Processing chenruohui-22-30-41-13

Processing wangaifang-13-19-28-74
Processing wangbaoyu-7-12-21-75
Processing wangbingdao-26-36-55-132
Processing wangdahua-19-24-33-89
Processing wangdandan-32-39-50-142
Processing wangdayang-3-14-41-137
Processing wangdehao-40-51-65-159
Processing wangdeqian-26-38-60-156
Processing wangfeng-14-19-31-85
Processing wangguojian-20-25-39-86
Processing wangguolong-16-21-33-84
Processing wanghongchao-33-42-69-153
Processing wanghongchu-23-35-67-142
Processing wangjiajin-35-46-69-148
Processing wangjian-17-24-32-88
Processing wangjinlie-26-33-45-87
Processing wangkehuai-47-57-80-157
Processing wangligang-25-36-55-154
Processing wangluhua-18-23-32-87
Processing wangmingxuan-35-45-67-157
Processing wangronglin-16-21-28-81
Processing wangshuilin-23-35-53-134
Processing wangsonglin-19-24-27-86
Processing wangwei-32-39-50-139
Processing wangwuzhuang-28-33-41-98
Processing wangxinhui-18-23-31-85
Processing wangyeping-25-33-49-129
Processing wangyunxian-18-27-39-134
Processing wangyuzhi-29-38-55-15

In [37]:
# 切出主动脉
def find_coordinate(height, width, label_file, aorta):
    with open(label_file, 'r') as f:
        lines = f.readlines()
    assert len(lines) <= 2, f'label.txt应该存储不多于2个label：{label_file.split("/")[-1]}'
    if len(lines) == 1:
        assert aorta == 'j', f'如果只有一个label那么此时应为降主动脉, 但实际为{aorta}：{label_file.split("/")[-1]}'
        corr = list(map(lambda x: float(x), lines[0].split()))
        x, y, w, h = corr[1], corr[2], corr[3], corr[4]
        assert 0.25 < x < 0.75 and 0.25 < y < 0.75, f'边界框中心({x}, {y})出界：{label_file.split("/")[-1]}'
    else:
        corr1, corr2 = list(map(lambda x: float(x), lines[0].split())), list(map(lambda x: float(x), lines[1].split()))
        assert 0.25 < corr1[1] < 0.75 and 0.25 < corr1[2] < 0.75, f'边界框1中心({corr1[1]}, {corr1[2]})出界：{label_file.split("/")[-1]}'
        assert 0.25 < corr2[1] < 0.75 and 0.25 < corr2[2] < 0.75, f'边界框2中心({corr2[1]}, {corr2[2]})出界：{label_file.split("/")[-1]}'
        if aorta == 's':
            x, y, w, h = (corr1[1], corr1[2], corr1[3], corr1[4]) if corr1[2] < corr2[2] else (corr2[1], corr2[2], corr2[3], corr2[4])
        elif aorta == 'j':
            x, y, w, h = (corr1[1], corr1[2], corr1[3], corr1[4]) if corr1[2] > corr2[2] else (corr2[1], corr2[2], corr2[3], corr2[4])
        else:
            raise Exception(f'aorta 应该为"s"或"j"其中之一: {label_file.split("/")[-1]}')
    w, h = int(width*w), int(height*h)
    w, h = max(w, h), max(w, h)
    return int(width*x-w/2), int(height*y-h/2), int(width*x+w/2+1), int(height*y+h/2+1)

def crop_images(input_path, error_patient_list):
    for patient in sorted(os.listdir(input_path)):
        print(f'******Processing {patient}******')
        image_path = os.path.join(input_path, patient, 'images')
        label_path = os.path.join(input_path, patient, 'labels')
        crop_path = os.path.join(input_path, patient, 'crops')
        if os.path.exists(crop_path):
            shutil.rmtree(crop_path)
        os.mkdir(crop_path)
        pl = patient.split('-')
        aorta_start, branch_start, branch_end, aorta_end = int(pl[1])-1, int(pl[2])-1, int(pl[3])-1, int(pl[4])-1
        crop_flag = True
        offset = branch_end - branch_start
        start, end = branch_start + int(0.1*offset), branch_end - int(0.2*offset)
        for i in range(start, end):
            img = Image.open(os.path.join(image_path, f'{patient}_{i:04d}.png'))
            img = np.array(img)
            try:
                x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{i:04d}.txt'), 's')
            except:
                traceback.print_exc()
                crop_flag = False
            else:#if crop_flag:
                crop = img[y1:y2, x1:x2]
                crop = Image.fromarray(crop)
                crop.save(os.path.join(crop_path, f'{patient}_s_{i:04d}.png'))
            try:
                x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{i:04d}.txt'), 'j')
            except:
                traceback.print_exc()
                crop_flag = False
            else:#if crop_flag:
                crop = img[y1:y2, x1:x2]
                crop = Image.fromarray(crop)
                crop.save(os.path.join(crop_path, f'{patient}_j_{i:04d}.png'))
        offset = aorta_end - branch_end
        start, end = branch_end + int(0.1*offset), aorta_end - int(0.2*offset)
        for i in range(start, end):
            img = Image.open(os.path.join(image_path, f'{patient}_{i:04d}.png'))
            img = np.array(img)
            try:
                x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{i:04d}.txt'), 'j')
            except:
                traceback.print_exc()
                crop_flag = False
            else:#if crop_flag:
                crop = img[y1:y2, x1:x2]
                crop = Image.fromarray(crop)
                crop.save(os.path.join(crop_path, f'{patient}_j_{i:04d}.png')) 
        if not crop_flag:
            #print('Delete crop_path')
            #shutil.rmtree(crop_path)
            error_patient_list.append(patient)
            
epl1 = []
#crop_images('/nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative', epl1)
crop_images('/nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2', epl1)

******Processing baimeizhu-27-37-54-136******


Traceback (most recent call last):
  File "<ipython-input-37-508357bc551f>", line 43, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{i:04d}.txt'), 's')
  File "<ipython-input-37-508357bc551f>", line 3, in find_coordinate
    with open(label_file, 'r') as f:
FileNotFoundError: [Errno 2] No such file or directory: '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/baimeizhu-27-37-54-136/labels/baimeizhu-27-37-54-136_0037.txt'
Traceback (most recent call last):
  File "<ipython-input-37-508357bc551f>", line 52, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{i:04d}.txt'), 'j')
  File "<ipython-input-37-508357bc551f>", line 3, in find_coordinate
    with open(label_file, 'r') as f:
FileNotFoundError: [Errno 2] No such file or directory: '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/baimeizhu-27-37-54-136/labels/baimeizhu-27-37-54-136_0037.txt'
Traceback (most rece

******Processing bairuixin-13-18-35-81******
******Processing baodezhong-22-29-39-91******
******Processing baoxialin-23-34-57-158******
******Processing baoyanzu-15-20-31-87******
******Processing caijinfei-34-42-51-155******


Traceback (most recent call last):
  File "<ipython-input-37-508357bc551f>", line 43, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{i:04d}.txt'), 's')
  File "<ipython-input-37-508357bc551f>", line 7, in find_coordinate
    assert aorta == 'j', f'如果只有一个label那么此时应为降主动脉, 但实际为{aorta}：{label_file.split("/")[-1]}'
AssertionError: 如果只有一个label那么此时应为降主动脉, 但实际为s：caijinfei-34-42-51-155_0042.txt
Traceback (most recent call last):
  File "<ipython-input-37-508357bc551f>", line 43, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{i:04d}.txt'), 's')
  File "<ipython-input-37-508357bc551f>", line 7, in find_coordinate
    assert aorta == 'j', f'如果只有一个label那么此时应为降主动脉, 但实际为{aorta}：{label_file.split("/")[-1]}'
AssertionError: 如果只有一个label那么此时应为降主动脉, 但实际为s：caijinfei-34-42-51-155_0044.txt
Traceback (most recent call last):
  File "<ipython-input-37-508357bc551f>", line 43, in crop_images
 

Traceback (most recent call last):
  File "<ipython-input-37-508357bc551f>", line 66, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{i:04d}.txt'), 'j')
  File "<ipython-input-37-508357bc551f>", line 3, in find_coordinate
    with open(label_file, 'r') as f:
FileNotFoundError: [Errno 2] No such file or directory: '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/caijinfei-34-42-51-155/labels/caijinfei-34-42-51-155_0124.txt'
Traceback (most recent call last):
  File "<ipython-input-37-508357bc551f>", line 66, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{i:04d}.txt'), 'j')
  File "<ipython-input-37-508357bc551f>", line 3, in find_coordinate
    with open(label_file, 'r') as f:
FileNotFoundError: [Errno 2] No such file or directory: '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/caijinfei-34-42-51-155/labels/caijinfei-34-42-51-155_0125.txt'
Traceback (most rece

...

******Processing zhouxianhang-13-17-26-78******


Traceback (most recent call last):
  File "<ipython-input-37-508357bc551f>", line 43, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{i:04d}.txt'), 's')
  File "<ipython-input-37-508357bc551f>", line 3, in find_coordinate
    with open(label_file, 'r') as f:
FileNotFoundError: [Errno 2] No such file or directory: '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/zhouxianhang-13-17-26-78/labels/zhouxianhang-13-17-26-78_0016.txt'
Traceback (most recent call last):
  File "<ipython-input-37-508357bc551f>", line 52, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{i:04d}.txt'), 'j')
  File "<ipython-input-37-508357bc551f>", line 3, in find_coordinate
    with open(label_file, 'r') as f:
FileNotFoundError: [Errno 2] No such file or directory: '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/zhouxianhang-13-17-26-78/labels/zhouxianhang-13-17-26-78_0016.txt'


******Processing zhouxincang-31-42-59-139******
******Processing zhouyixiao-22-28-40-83******
******Processing zhouyongyan-21-33-51-136******
******Processing zhouzhangnv-12-18-31-80******


Traceback (most recent call last):
  File "<ipython-input-37-508357bc551f>", line 43, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{i:04d}.txt'), 's')
  File "<ipython-input-37-508357bc551f>", line 7, in find_coordinate
    assert aorta == 'j', f'如果只有一个label那么此时应为降主动脉, 但实际为{aorta}：{label_file.split("/")[-1]}'
AssertionError: 如果只有一个label那么此时应为降主动脉, 但实际为s：zhouzhangnv-12-18-31-80_0027.txt


******Processing zhucaiqin-20-31-50-124******
******Processing zhuguizhou-18-23-34-85******
******Processing zhuguofang-15-20-32-85******
******Processing zhujianfu-16-22-28-87******
******Processing zhujianfu-30-42-62-154******
******Processing zhukuanwen-15-21-29-83******
******Processing zhuodepeng-9-14-23-78******
******Processing zhuxingfu-14-19-32-82******
******Processing zhuyanjie-20-25-35-89******


In [38]:
print(len(epl1))
print(epl1)

115
['baimeizhu-27-37-54-136', 'caijinfei-34-42-51-155', 'chengenye-13-18-28-79', 'chenguoming-28-38-60-144', 'chenjinfu-11-18-30-80', 'chenjunbing-22-29-34-89', 'chenliuer-23-32-43-140', 'chenpeisheng-15-20-28-82', 'chenweiming-29-39-59-150', 'chenyilin-21-26-36-86', 'dengdeyu-18-27-43-152', 'dingshuigen-28-38-53-136', 'fangjinchai-19-23-31-81', 'fengaiyuan-26-32-46-121', 'fengqiufen-14-19-27-75', 'fujingjing-24-32-39-127', 'gaoanhui-17-22-31-85', 'guohao-25-30-38-94', 'guoyafang-24-29-52-136', 'hebaomin-28-37-54-145', 'hongxia-17-34-43-122', 'huangjiabao-22-27-33-53', 'huaweiqin-15-25-33-77', 'huxiangxin-19-24-34-86', 'huyongxian-18-25-38-88', 'jiangchengsan-25-34-50-133', 'jiangguanshui-16-21-31-85', 'jiangjiaqi-23-30-39-140', 'jiangjinfeng-26-39-58-152', 'jiangxiangzhong-15-20-29-83', 'jinjianrong-19-24-32-86', 'jinxianqing-15-23-37-81', 'jinyu-19-29-44-146', 'laijinhu-14-21-34-83', 'liaoxiuhua-36-44-55-144', 'lijunhua-15-21-27-82', 'lintianye-18-28-46-136', 'liujun-8-15-27-95', 'l

In [ ]:
# # 将切出的主动脉移动到一起
# def move_to_classify(input_root, output_path):
#     if not os.path.exists(output_path):
#         os.makedirs(output_path)
    
#     for patient in os.listdir(input_root):
#         if os.path.isfile(os.path.join(input_root, patient)):
#             continue
#         print(f'****Processing {patient}****')
#         for file in os.listdir(os.path.join(input_root, patient, 'crops')):
#             shutil.copy(os.path.join(input_root, patient, 'crops', file), output_path)
            
# move_to_classify('/nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/', f'/nfs3-p1/zsxm/dataset/aorta_classify_ct_{lower_b}_{upper_b}/0/')

## 3.固定训练集和测试集

### 3.1 阳性数据

In [5]:
#patient_roots = ['/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive','/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive_new/']
patient_roots = ['/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive_new2/']
patient_list = []
for patient_root in patient_roots:
    for patient in os.listdir(patient_root):
        if os.path.isdir(os.path.join(patient_root, patient)):
            patient_list.append(os.path.join(patient_root, patient))

In [6]:
random.seed(7987)
val_patient_list = random.sample(patient_list, int(len(patient_list)*0.2))
train_patient_list = [p for p in patient_list if p not in val_patient_list]#list(set(patient_list) - set(val_patient_list))
sorted(train_patient_list)
sorted(val_patient_list)
print(len(train_patient_list), len(val_patient_list), len(train_patient_list)+len(val_patient_list))

42 10 52


In [7]:
def move_to_classify(patient_list, output_path):
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    for patient in patient_list:
        print(f'****Processing {patient}****')
        for file in os.listdir(os.path.join(patient, 'crops')):
            shutil.copy(os.path.join(patient, 'crops', file), output_path)
            
move_to_classify(train_patient_list, f'/nfs3-p1/zsxm/dataset/aorta_classify_ct_{lower_b}_{upper_b}/train/1/')
move_to_classify(val_patient_list, f'/nfs3-p1/zsxm/dataset/aorta_classify_ct_{lower_b}_{upper_b}/val/1/')

****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive_new2/baochunsheng-J-20-82****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive_new2/baojinan-J-24-87****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive_new2/chengyonghui-J-17-81****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive_new2/chensiqi-S-17-28-J-17-69****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive_new2/dukun-S-20-22-J-20-23****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive_new2/fanghongwei-J-22-71****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive_new2/fangyuxian-J-23-60****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive_new2/guoyinhua-J-13-87****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive_new2/heguangfa-J-27-88****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive_new2/huanglijun-J-24-98****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/posi

### 3.2阴性数据

In [40]:
#patient_roots = ['/nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/']
patient_roots = ['/nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/']
patient_list = []
for patient_root in patient_roots:
    for patient in os.listdir(patient_root):
        if os.path.isdir(os.path.join(patient_root, patient)):
            patient_list.append(os.path.join(patient_root, patient))

In [41]:
random.seed(7888)
val_patient_list = random.sample(patient_list, int(len(patient_list)*0.2))
train_patient_list = [p for p in patient_list if p not in val_patient_list]#list(set(patient_list) - set(val_patient_list))
sorted(train_patient_list)
sorted(val_patient_list)
print(len(train_patient_list), len(val_patient_list), len(train_patient_list)+len(val_patient_list))

314 78 392


In [42]:
def move_to_classify(patient_list, output_path):
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    for patient in patient_list:
        print(f'****Processing {patient}****')
        for file in os.listdir(os.path.join(patient, 'crops')):
            shutil.copy(os.path.join(patient, 'crops', file), output_path)
            
move_to_classify(train_patient_list, f'/nfs3-p1/zsxm/dataset/aorta_classify_ct_{lower_b}_{upper_b}/train/0/')
move_to_classify(val_patient_list, f'/nfs3-p1/zsxm/dataset/aorta_classify_ct_{lower_b}_{upper_b}/val/0/')

****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/zhangqingrong-32-43-73-161****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/baimeizhu-27-37-54-136****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/bairuixin-13-18-35-81****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/baodezhong-22-29-39-91****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/baoxialin-23-34-57-158****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/caijinfei-34-42-51-155****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/caishiping-18-24-34-85****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/caoyuying-18-22-33-79****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/chenbin-20-26-35-85****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/chenbingrong-31-40-61-153****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/ne

****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/jinruiyue-13-18-28-80****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/jinshuigen-18-23-36-79****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/jinxianqing-15-23-37-81****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/jinyongfa-25-29-37-92****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/jinyu-19-29-44-146****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/jinzhaoxiang-28-37-55-126****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/kerengang-25-30-41-91****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/kongxiangang-24-28-38-97****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/kongxiangwen-12-18-32-86****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/kongxiuchun-41-52-75-140****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/n

****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/songzongwei-20-26-39-94****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/sunlutao-14-23-70-127****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/sunyunxian-11-17-23-75****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/tangchunhua-10-16-20-76****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/tangshuiming-33-44-68-153****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/taofeng-18-22-28-81****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/taonan-19-24-37-87****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/taowenming-23-32-59-138****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/tongbo-25-35-51-153****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/tongchichang-16-21-35-88****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negativ

****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/yupengjie-26-34-46-134****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/yushengfu-31-40-56-154****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/yuweiliang-29-35-47-104****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/yuxiaotu-20-25-36-90****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/zhangbaoyou-29-40-60-148****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/zhangdanye-31-38-50-143****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/zhangduofeng-12-17-28-55****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/zhangguofeng-21-27-35-90****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/zhanghanmin-13-19-27-76****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/zhanghuanquan-17-22-29-75****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_

****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/panandan-16-21-31-81****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/yanchunhong-36-45-64-155****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/zhangfagen-22-27-37-92****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/maowenli-17-22-32-76****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/shenfabiao-29-39-61-135****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/chenweiqian-22-31-45-131****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/wangguolong-16-21-33-84****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/zengchunsheng-23-30-41-128****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/wangshuilin-23-35-53-134****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative_2/luyuefeng-24-28-40-94****
****Processing /nfs3-p1/zsxm/dataset/aorta_ct_img_lab

## 4.检查并统计crop图像尺寸

In [44]:
import os
from PIL import Image
import matplotlib.pyplot as plt
from numpy.lib.function_base import median

paths = [f'/nfs3-p1/zsxm/dataset/aorta_classify_ct_{lower_b}_{upper_b}/train/0/',
         f'/nfs3-p1/zsxm/dataset/aorta_classify_ct_{lower_b}_{upper_b}/val/0/',
         f'/nfs3-p1/zsxm/dataset/aorta_classify_ct_{lower_b}_{upper_b}/train/1/',
         f'/nfs3-p1/zsxm/dataset/aorta_classify_ct_{lower_b}_{upper_b}/val/1/']

size_list = []

count = 0
for path in paths:
    for f in os.listdir(path):
        img = Image.open(os.path.join(path, f))
        size_list.append(img.height)
        if img.height != img.width:
            print(f'{os.path.join(path, f)}: height:{img.height}, width:{img.width}')
            count += 1

size_list.sort()

# plt.hist(size_list, bins=100)
# plt.show()

sum = 0
hash_list = {}
for size in size_list:
    sum += size
    hash_list[size] = hash_list.get(size, 0) + 1

avg = sum / len(size_list)

key = len(size_list) // 2
med = (size_list[key] + size_list[-key-1]) / 2

v_list = list(hash_list.items())
v_list.sort(key=lambda x: x[1], reverse=True)
mode = v_list[0][0]

print(f'avg:{avg}, median:{med}, mode:{mode}, h_w_not_equal:{count}')

avg:45.0381723904238, median:42.0, mode:41, h_w_not_equal:0
